In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob 
import os
import warnings
import networkx as nx
import matplotlib.colors as mcolors
warnings.simplefilter(action='ignore', category=FutureWarning)
from scipy.stats import pearsonr
from sklearn.covariance import GraphicalLassoCV
from networkx.generators.community import gaussian_random_partition_graph
import scipy.stats as stats
from scipy.stats import ttest_ind
import nibabel as nib 
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
pd.options.display.max_rows = None
pd.options.display.max_columns = None
import glob as glob
import math
#set random generator outside loop 
#seed everytrhing once at begibning and dont touch it 
np.random.seed(3)

#### For tau graphs, just do these steps, but on Tau suvr, stratify by amyloid quartile 

- For stratifying by amyloid status,  Royse 2021 Validation of PET positivity thresholds in centiloids established that 21 (or 20) is about so use this

#### Data has been preprocessed into amyloid quartiles for both adni and A4

## I need to verify they used the same freesurfer atlases (for region specific analyses)


#### General steps 
#### 1. A. Read ADNI data quartile csvs 
- These data have regional tau SUVRs, centiloid values, visit dates, and ID 
- (103, 228) in low 
- (107, 228) in med low 
- (100, 228) in med high 
- (101, 228) in high 

In [10]:
adni_cent_low = pd.read_csv('./adni/adni_amy_tau_merged_cent_low_quartile.csv')
adni_cent_med_low = pd.read_csv('./adni/adni_amy_tau_merged_cent_med_low_quartile.csv')
adni_cent_med_high = pd.read_csv('./adni/adni_amy_tau_merged_cent_med_high_quartile.csv')
adni_cent_high = pd.read_csv('./adni/adni_amy_tau_merged_cent_high_quartile.csv')

In [14]:
adni_cent_low.head(5)

,RID,SCANDATE,TRACER,CENTILOIDS,VISCODE,VISCODE2,EXAMDATE,META_TEMPORAL_SUVR,META_TEMPORAL_VOLUME,CTX_ENTORHINAL_SUVR,CTX_ENTORHINAL_VOLUME,INFERIOR_CEREBGM_SUVR,INFERIOR_CEREBGM_VOLUME,HEMIWM_SUVR,HEMIWM_VOLUME,BRAIN_STEM_SUVR,BRAIN_STEM_VOLUME,CHOROID_SUVR,CHOROID_VOLUME,MIDDLEFR_SUVR,MIDDLEFR_VOLUME,ORBITOFR_SUVR,ORBITOFR_VOLUME,PARSFR_SUVR,PARSFR_VOLUME,CTX_BANKSSTS_SUVR,CTX_BANKSSTS_VOLUME,CTX_CAUDALANTERIORCINGULATE_SUVR,CTX_CAUDALANTERIORCINGULATE_VOLUME,CTX_CUNEUS_SUVR,CTX_CUNEUS_VOLUME,CTX_FUSIFORM_SUVR,CTX_FUSIFORM_VOLUME,CTX_INFERIORPARIETAL_SUVR,CTX_INFERIORPARIETAL_VOLUME,CTX_INFERIORTEMPORAL_SUVR,CTX_INFERIORTEMPORAL_VOLUME,CTX_INSULA_SUVR,CTX_INSULA_VOLUME,CTX_ISTHMUSCINGULATE_SUVR,CTX_ISTHMUSCINGULATE_VOLUME,CTX_LATERALOCCIPITAL_SUVR,CTX_LATERALOCCIPITAL_VOLUME,CTX_LINGUAL_SUVR,CTX_LINGUAL_VOLUME,CTX_MIDDLETEMPORAL_SUVR,CTX_MIDDLETEMPORAL_VOLUME,CTX_PARACENTRAL_SUVR,CTX_PARACENTRAL_VOLUME,CTX_PARAHIPPOCAMPAL_SUVR,CTX_PARAHIPPOCAMPAL_VOLUME,CTX_PERICALCARINE_SUVR,CTX_PERICALCARINE_VOLUME,CTX_POSTCENTRAL_SUVR,CTX_POSTCENTRAL_VOLUME,CTX_POSTERIORCINGULATE_SUVR,CTX_POSTERIORCINGULATE_VOLUME,CTX_PRECENTRAL_SUVR,CTX_PRECENTRAL_VOLUME,CTX_PRECUNEUS_SUVR,CTX_PRECUNEUS_VOLUME,CTX_ROSTRALANTERIORCINGULATE_SUVR,CTX_ROSTRALANTERIORCINGULATE_VOLUME,CTX_SUPERIORFRONTAL_SUVR,CTX_SUPERIORFRONTAL_VOLUME,CTX_SUPERIORPARIETAL_SUVR,CTX_SUPERIORPARIETAL_VOLUME,CTX_SUPERIORTEMPORAL_SUVR,CTX_SUPERIORTEMPORAL_VOLUME,CTX_SUPRAMARGINAL_SUVR,CTX_SUPRAMARGINAL_VOLUME,CTX_TEMPORALPOLE_SUVR,CTX_TEMPORALPOLE_VOLUME,CTX_TRANSVERSETEMPORAL_SUVR,CTX_TRANSVERSETEMPORAL_VOLUME,ACCUMBENS_AREA_SUVR,ACCUMBENS_AREA_VOLUME,AMYGDALA_SUVR,AMYGDALA_VOLUME,CAUDATE_SUVR,CAUDATE_VOLUME,PALLIDUM_SUVR,PALLIDUM_VOLUME,PUTAMEN_SUVR,PUTAMEN_VOLUME,THALAMUS_PROPER_SUVR,THALAMUS_PROPER_VOLUME,LEFT_MIDDLEFR_SUVR,LEFT_MIDDLEFR_VOLUME,LEFT_ORBITOFR_SUVR,LEFT_ORBITOFR_VOLUME,LEFT_PARSFR_SUVR,LEFT_PARSFR_VOLUME,RIGHT_MIDDLEFR_SUVR,RIGHT_MIDDLEFR_VOLUME,RIGHT_ORBITOFR_SUVR,RIGHT_ORBITOFR_VOLUME,RIGHT_PARSFR_SUVR,RIGHT_PARSFR_VOLUME,CTX_LH_BANKSSTS_SUVR,CTX_LH_BANKSSTS_VOLUME,CTX_LH_CAUDALANTERIORCINGULATE_SUVR,CTX_LH_CAUDALANTERIORCINGULATE_VOLUME,CTX_LH_CUNEUS_SUVR,CTX_LH_CUNEUS_VOLUME,CTX_LH_ENTORHINAL_SUVR,CTX_LH_ENTORHINAL_VOLUME,CTX_LH_FUSIFORM_SUVR,CTX_LH_FUSIFORM_VOLUME,CTX_LH_INFERIORPARIETAL_SUVR,CTX_LH_INFERIORPARIETAL_VOLUME,CTX_LH_INFERIORTEMPORAL_SUVR,CTX_LH_INFERIORTEMPORAL_VOLUME,CTX_LH_INSULA_SUVR,CTX_LH_INSULA_VOLUME,CTX_LH_ISTHMUSCINGULATE_SUVR,CTX_LH_ISTHMUSCINGULATE_VOLUME,CTX_LH_LATERALOCCIPITAL_SUVR,CTX_LH_LATERALOCCIPITAL_VOLUME,CTX_LH_LINGUAL_SUVR,CTX_LH_LINGUAL_VOLUME,CTX_LH_MIDDLETEMPORAL_SUVR,CTX_LH_MIDDLETEMPORAL_VOLUME,CTX_LH_PARACENTRAL_SUVR,CTX_LH_PARACENTRAL_VOLUME,CTX_LH_PARAHIPPOCAMPAL_SUVR,CTX_LH_PARAHIPPOCAMPAL_VOLUME,CTX_LH_PERICALCARINE_SUVR,CTX_LH_PERICALCARINE_VOLUME,CTX_LH_POSTCENTRAL_SUVR,CTX_LH_POSTCENTRAL_VOLUME,CTX_LH_POSTERIORCINGULATE_SUVR,CTX_LH_POSTERIORCINGULATE_VOLUME,CTX_LH_PRECENTRAL_SUVR,CTX_LH_PRECENTRAL_VOLUME,CTX_LH_PRECUNEUS_SUVR,CTX_LH_PRECUNEUS_VOLUME,CTX_LH_ROSTRALANTERIORCINGULATE_SUVR,CTX_LH_ROSTRALANTERIORCINGULATE_VOLUME,CTX_LH_SUPERIORFRONTAL_SUVR,CTX_LH_SUPERIORFRONTAL_VOLUME,CTX_LH_SUPERIORPARIETAL_SUVR,CTX_LH_SUPERIORPARIETAL_VOLUME,CTX_LH_SUPERIORTEMPORAL_SUVR,CTX_LH_SUPERIORTEMPORAL_VOLUME,CTX_LH_SUPRAMARGINAL_SUVR,CTX_LH_SUPRAMARGINAL_VOLUME,CTX_LH_TEMPORALPOLE_SUVR,CTX_LH_TEMPORALPOLE_VOLUME,CTX_LH_TRANSVERSETEMPORAL_SUVR,CTX_LH_TRANSVERSETEMPORAL_VOLUME,CTX_RH_BANKSSTS_SUVR,CTX_RH_BANKSSTS_VOLUME,CTX_RH_CAUDALANTERIORCINGULATE_SUVR,CTX_RH_CAUDALANTERIORCINGULATE_VOLUME,CTX_RH_CUNEUS_SUVR,CTX_RH_CUNEUS_VOLUME,CTX_RH_ENTORHINAL_SUVR,CTX_RH_ENTORHINAL_VOLUME,CTX_RH_FUSIFORM_SUVR,CTX_RH_FUSIFORM_VOLUME,CTX_RH_INFERIORPARIETAL_SUVR,CTX_RH_INFERIORPARIETAL_VOLUME,CTX_RH_INFERIORTEMPORAL_SUVR,CTX_RH_INFERIORTEMPORAL_VOLUME,CTX_RH_INSULA_SUVR,CTX_RH_INSULA_VOLUME,CTX_RH_ISTHMUSCINGULATE_SUVR,CTX_RH_ISTHMUSCINGULATE_VOLUME,CTX_RH_LATERALOCCIPITAL_SUVR,CTX_RH_LATERALOCCIPITAL_VOLUME,CTX_RH_LINGUAL_SUVR,CTX_

#### 1. B. Read A4 data 
- These data have regional tau SUVRs, centiloid values, visit dates, and ID 
- (101, 277) in low 
- (86, 277) in med low 
- (94, 277) in med high
- (92, 277) in high

In [15]:
a4_cent_low = pd.read_csv('./a4/a4_amy_tau_merged_cent_low_quartile.csv')
a4_cent_med_low = pd.read_csv('./a4/a4_amy_tau_merged_cent_med_low_quartile.csv')
a4_cent_med_high = pd.read_csv('./a4/a4_amy_tau_merged_cent_med_high_quartile.csv')
a4_cent_high = pd.read_csv('./a4/a4_adni_amy_tau_merged_cent_high_quartile.csv')

In [17]:
a4_cent_low.head(5)

,BID,update_stamp_x,ligand,centiloid,Mean_3rd_Ventricle,Mean_4th_Ventricle,Mean_Brain_Stem,Mean_CC_Anterior,Mean_CC_Central,Mean_CC_Mid_Anterior,Mean_CC_Mid_Posterior,Mean_CC_Posterior,Mean_CSF,Mean_Left_Accumbens_area,Mean_Left_Amygdala,Mean_Left_Caudate,Mean_Left_Cerebellum_Cortex,Mean_Left_Cerebellum_White_Matter,Mean_Left_Cerebral_White_Matter,Mean_Left_Hippocampus,Mean_Left_Inf_Lat_Vent,Mean_Left_Lateral_Ventricle,Mean_Left_Pallidum,Mean_Left_Putamen,Mean_Left_Thalamus,Mean_Left_VentralDC,Mean_Left_choroid_plexus,Mean_Left_vessel,Mean_Optic_Chiasm,Mean_Right_Accumbens_area,Mean_Right_Amygdala,Mean_Right_Caudate,Mean_Right_Cerebellum_Cortex,Mean_Right_Cerebellum_White_Matter,Mean_Right_Cerebral_White_Matter,Mean_Right_Hippocampus,Mean_Right_Inf_Lat_Vent,Mean_Right_Lateral_Ventricle,Mean_Right_Pallidum,Mean_Right_Putamen,Mean_Right_Thalamus,Mean_Right_VentralDC,Mean_Right_choroid_plexus,Mean_Right_vessel,Mean_Unknown,Mean_WM_hypointensities,Mean_ctx_lh_bankssts,Mean_ctx_lh_caudalanteriorcingulate,Mean_ctx_lh_caudalmiddlefrontal,Mean_ctx_lh_cuneus,Mean_ctx_lh_entorhinal,Mean_ctx_lh_frontalpole,Mean_ctx_lh_fusiform,Mean_ctx_lh_inferiorparietal,Mean_ctx_lh_inferiortemporal,Mean_ctx_lh_insula,Mean_ctx_lh_isthmuscingulate,Mean_ctx_lh_lateraloccipital,Mean_ctx_lh_lateralorbitofrontal,Mean_ctx_lh_lingual,Mean_ctx_lh_medialorbitofrontal,Mean_ctx_lh_middletemporal,Mean_ctx_lh_paracentral,Mean_ctx_lh_parahippocampal,Mean_ctx_lh_parsopercularis,Mean_ctx_lh_parsorbitalis,Mean_ctx_lh_parstriangularis,Mean_ctx_lh_pericalcarine,Mean_ctx_lh_postcentral,Mean_ctx_lh_posteriorcingulate,Mean_ctx_lh_precentral,Mean_ctx_lh_precuneus,Mean_ctx_lh_rostralanteriorcingulate,Mean_ctx_lh_rostralmiddlefrontal,Mean_ctx_lh_superiorfrontal,Mean_ctx_lh_superiorparietal,Mean_ctx_lh_superiortemporal,Mean_ctx_lh_supramarginal,Mean_ctx_lh_temporalpole,Mean_ctx_lh_transversetemporal,Mean_ctx_rh_bankssts,Mean_ctx_rh_caudalanteriorcingulate,Mean_ctx_rh_caudalmiddlefrontal,Mean_ctx_rh_cuneus,Mean_ctx_rh_entorhinal,Mean_ctx_rh_frontalpole,Mean_ctx_rh_fusiform,Mean_ctx_rh_inferiorparietal,Mean_ctx_rh_inferiortemporal,Mean_ctx_rh_insula,Mean_ctx_rh_isthmuscingulate,Mean_ctx_rh_lateraloccipital,Mean_ctx_rh_lateralorbitofrontal,Mean_ctx_rh_lingual,Mean_ctx_rh_medialorbitofrontal,Mean_ctx_rh_middletemporal,Mean_ctx_rh_paracentral,Mean_ctx_rh_parahippocampal,Mean_ctx_rh_parsopercularis,Mean_ctx_rh_parsorbitalis,Mean_ctx_rh_parstriangularis,Mean_ctx_rh_pericalcarine,Mean_ctx_rh_postcentral,Mean_ctx_rh_posteriorcingulate,Mean_ctx_rh_precentral,Mean_ctx_rh_precuneus,Mean_ctx_rh_rostralanteriorcingulate,Mean_ctx_rh_rostralmiddlefrontal,Mean_ctx_rh_superiorfrontal,Mean_ctx_rh_superiorparietal,Mean_ctx_rh_superiortemporal,Mean_ctx_rh_supramarginal,Mean_ctx_rh_temporalpole,Mean_ctx_rh_transversetemporal,Mean_non_WM_hypointensities,Volume_mm3_3rd_Ventricle,Volume_mm3_4th_Ventricle,Volume_mm3_Brain_Stem,Volume_mm3_CC_Anterior,Volume_mm3_CC_Central,Volume_mm3_CC_Mid_Anterior,Volume_mm3_CC_Mid_Posterior,Volume_mm3_CC_Posterior,Volume_mm3_CSF,Volume_mm3_Left_Accumbens_area,Volume_mm3_Left_Amygdala,Volume_mm3_Left_Caudate,Volume_mm3_Left_Cerebellum_Cortex,Volume_mm3_Left_Cerebellum_White_Matter,Volume_mm3_Left_Cerebral_White_Matter,Volume_mm3_Left_Hippocampus,Volume_mm3_Left_Inf_Lat_Vent,Volume_mm3_Left_Lateral_Ventricle,Volume_mm3_Left_Pallidum,Volume_mm3_Left_Putamen,Volume_mm3_Left_Thalamus,Volume_mm3_Left_VentralDC,Volume_mm3_Left_choroid_plexus,Volume_mm3_Left_vessel,Volume_mm3_Optic_Chiasm,Volume_mm3_Right_Accumbens_area,Volume_mm3_Right_Amygdala,Volume_mm3_Right_Caudate,Volume_mm3_Right_Cerebellum_Cortex,Volume_mm3_Right_Cerebellum_White_Matter,Volume_mm3_Right_Cerebral_White_Matter,Volume_mm3_Right_Hippocampus,Volume_mm3_Right_Inf_Lat_Vent,Volume_mm3_Right_Lateral_Ventricle,Volume_mm3_Right_Pallidum,Volume_mm3_Right_Putamen,Volume_mm3_Right_Thalamus,Volume_mm3_Right_VentralDC,Volume_mm3_Right_choroid_plexus,Volume_mm3_Right_vessel,Volume_mm3_Unknown,Volume_mm3_WM_hypointensiti

In [2]:
df = pd.read_csv("/home/mlauber/mci_mri_graph/construct_ggm/ml_dataframe.csv")

In [3]:
#look for and load in the A4 dataset because I think it has more patients

In [4]:
print(len(df))
print(df.columns)

793
Index(['RID', 'PTGENDER', 'PTEDUCAT', 'PTTLANG', 'PTETHCAT', 'PTRACCAT',
       'AGE_MRI', 'DIAGNOSIS', 'Q1SCORE', 'Q4SCORE',
       ...
       'MMSE3', 'MMSE4', 'MMSE5', 'MMSE6', 'MMSE7', 'META_TEMPORAL',
       'MESIAL_TEMPORAL', 'TEMPO_PARIETAL', 'FRONTAL', 'NEOCORTICAL'],
      dtype='object', length=202)


- out of 793 patients 
- if cut off is 24 then 
- 339 amy pos
- 454 amy neg

- if cut off is 40 then 
- 256 pos : if I do quartiles then this is 64 patients in each quartile, could be kinda small, maybe use this as the replication dataset if possible 
- 526 neg 

In [11]:
#separate into pos and neg 
amy_pos = df[df['CENTILOIDS']>= 18]
amy_neg = df[df['CENTILOIDS']< 18]

# #if I use a stricter cut off of 40
# amy_pos = df[df['CENTILOIDS']>= 40]
# amy_neg = df[df['CENTILOIDS']< 40]
print(len(amy_pos))
print(len(amy_neg))

371
422


In [12]:
371 / 4

92.75

#### 2. Run and train separate graphical models for amyloid+ and amyloid- populations

##### I. Prepare data steps:
- a. need drop all non brain region vol cols - keep only brain regions 
- also drop the left and right ones 

In [7]:
#do some reg ex search for _VOLUME
def filter_regions(df):
    filtered_df = df.filter(regex='_VOLUME$')
    return filtered_df

amy_pos_reg = filter_regions(amy_pos)
amy_neg_reg = filter_regions(amy_neg)
amy_pos_reg.to_csv('amy_pos_regions.csv')
amy_neg_reg.to_csv('amy_neg_reg.csv')
print(amy_neg_reg.columns)

Index(['META_TEMPORAL_VOLUME', 'CTX_ENTORHINAL_VOLUME',
       'INFERIOR_CEREBGM_VOLUME', 'HEMIWM_VOLUME', 'BRAIN_STEM_VOLUME',
       'CHOROID_VOLUME', 'MIDDLEFR_VOLUME', 'ORBITOFR_VOLUME', 'PARSFR_VOLUME',
       'CTX_BANKSSTS_VOLUME',
       ...
       'LEFT_CAUDATE_VOLUME', 'LEFT_PALLIDUM_VOLUME', 'LEFT_PUTAMEN_VOLUME',
       'LEFT_THALAMUS_PROPER_VOLUME', 'RIGHT_ACCUMBENS_AREA_VOLUME',
       'RIGHT_AMYGDALA_VOLUME', 'RIGHT_CAUDATE_VOLUME',
       'RIGHT_PALLIDUM_VOLUME', 'RIGHT_PUTAMEN_VOLUME',
       'RIGHT_THALAMUS_PROPER_VOLUME'],
      dtype='object', length=110)


In [8]:
print(len(amy_pos.columns))

202


In [9]:
#drop left and right because some are redudant 
#something like if LEFT_ RIGHT_ or LH or RH
def filter_left_right(df):
    columns_to_keep = [col for col in df.columns if 'LEFT' not in col and 'RIGHT' not in col and 'LH' not in col and 'RH' not in col]
    return df[columns_to_keep]

# Assuming you have data in amy_pos_reg and amy_neg_reg
amy_pos_reg_comb = filter_left_right(amy_pos_reg)
amy_neg_reg_comb = filter_left_right(amy_neg_reg)

amy_pos_reg_comb.to_csv('amy_pos_reg_dropleftright.csv')
amy_neg_reg_comb.to_csv('amy_neg_reg_dropleftright.csv')

print(amy_pos_reg_comb.columns)
print(len(amy_pos_reg_comb.columns))



Index(['META_TEMPORAL_VOLUME', 'INFERIOR_CEREBGM_VOLUME', 'HEMIWM_VOLUME',
       'BRAIN_STEM_VOLUME', 'CHOROID_VOLUME', 'MIDDLEFR_VOLUME',
       'ORBITOFR_VOLUME', 'PARSFR_VOLUME', 'CTX_BANKSSTS_VOLUME',
       'CTX_CAUDALANTERIORCINGULATE_VOLUME', 'CTX_CUNEUS_VOLUME',
       'CTX_FUSIFORM_VOLUME', 'CTX_INFERIORPARIETAL_VOLUME',
       'CTX_INFERIORTEMPORAL_VOLUME', 'CTX_INSULA_VOLUME',
       'CTX_ISTHMUSCINGULATE_VOLUME', 'CTX_LATERALOCCIPITAL_VOLUME',
       'CTX_LINGUAL_VOLUME', 'CTX_MIDDLETEMPORAL_VOLUME',
       'CTX_PARACENTRAL_VOLUME', 'CTX_PARAHIPPOCAMPAL_VOLUME',
       'CTX_PERICALCARINE_VOLUME', 'CTX_POSTCENTRAL_VOLUME',
       'CTX_POSTERIORCINGULATE_VOLUME', 'CTX_PRECENTRAL_VOLUME',
       'CTX_PRECUNEUS_VOLUME', 'CTX_ROSTRALANTERIORCINGULATE_VOLUME',
       'CTX_SUPERIORFRONTAL_VOLUME', 'CTX_SUPERIORPARIETAL_VOLUME',
       'CTX_SUPERIORTEMPORAL_VOLUME', 'CTX_SUPRAMARGINAL_VOLUME',
       'CTX_TEMPORALPOLE_VOLUME', 'CTX_TRANSVERSETEMPORAL_VOLUME',
       'ACCUMBENS_ARE

- b. find out if there are any regions which have 0 volume or rows where there is no entry 

- c. normalize the data 

- d. check that there are no null values 

##### II. Create bootstrap samples and run model for amy pos

- a. create boot strap samples 

- b. run model 

- c. calc metrics 

##### III. Create bootstrap samples and run model for amy neg

- a. create boot strap samples 

- b. run model 

- c. calc metrics 

##### IV. Compare graph metrics using t test 

##### Later can Use GMM (2 components) to stratify patients into amyloid pos and neg 
- example code in mri_pet/pet_predicition_mldataframe.csv
- do thresholding on normal cognition 